# Cleaning and Passing data to csv files

In this notebook we document the process of cleaning and passing the data to csv files for the project analyzing open street map data of the city of Lima, Peru.

In the repository, it is possible to find the file "lima_sample.osm" which contains a small subset of the XML tree.

In [7]:
### Helper functions. This functions help us to take data from the XML document.

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow
from collections import Counter

OSM_FILE = "lima_peru.osm"  # Replace this with your osm file
SAMPLE_FILE = "lima_sample.osm"

k = 500 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [8]:
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [3]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

We start the process of analizing our data by checking the street names. This is useful to see if the data present some irregularities and to implement solutions of the observed problems. 

In [4]:
## Getting street names
def get_streets(element):
    """Helper function to get street names from data """
    for tag in element.findall('tag'):
        if tag.attrib['k'].startswith("addr:street"):
            street_name = tag.attrib['v']
            return street_name
        else:
            continue

In [5]:
street_sample = []
for element in get_element(SAMPLE_FILE, tags=('node', 'way', 'relation')):
    street = get_streets(element)
    if  street is not None:
        street_sample.append(street)

In [6]:
street_sample[:20]

['Los Cedros',
 'Coricancha',
 'Satelite',
 'Federico Villareal',
 'Avenida San Felipe',
 'Avenida de los Precursores',
 u'Alameda Coronel Jos\xe9 Andr\xe9s R\xe1zuri',
 u'Avenida de Tom\xe1s Marsano',
 u'Alameda P\xe9rez de Tudela',
 'Los Algarrobos',
 'Jiron Jose Galvez 255',
 'Avenida Giuseppe Garibaldi',
 'Las Caobas',
 'Avenida Alfredo Benavides',
 'Las Calezas',
 u'Jir\xf3n de la Uni\xf3n',
 'Avenida Carlos Alberto Izaguirre',
 '7 de Junio',
 u'Avenida Mara\xf1on',
 'Gozzoli Sur']

The first thing we can notice from this is that we might have problems trying to aggregate the data because some entities are with accents and some not. We decide to uniform the data by removing all accents. 

In [7]:
import unicodedata
def strip_accents(s):
    """Strips all the accents from unicode data"""
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def normalizing_attributes(encode_attrib):
    """Pass the data to the appropriate unicode format and then accents are removed"""
    if encode_attrib is not None:
        atrib_encode = encode_attrib.encode("utf-8")
        attr_cont = atrib_encode.decode("utf-8")
        return strip_accents(attr_cont)

In [8]:
#Checking if function works appropriately
#Checking first 20 entries
map(normalizing_attributes, street_sample)[:20]

[u'Los Cedros',
 u'Coricancha',
 u'Satelite',
 u'Federico Villareal',
 u'Avenida San Felipe',
 u'Avenida de los Precursores',
 u'Alameda Coronel Jose Andres Razuri',
 u'Avenida de Tomas Marsano',
 u'Alameda Perez de Tudela',
 u'Los Algarrobos',
 u'Jiron Jose Galvez 255',
 u'Avenida Giuseppe Garibaldi',
 u'Las Caobas',
 u'Avenida Alfredo Benavides',
 u'Las Calezas',
 u'Jiron de la Union',
 u'Avenida Carlos Alberto Izaguirre',
 u'7 de Junio',
 u'Avenida Maranon',
 u'Gozzoli Sur']

Now that we have uniformly treated the names of the streets, we want to check further problems with the data. Note that the first name of the street names gives us its type (e.g. Avenida, Jiron, Pasaje). With this information, we want to chek if there are certain problems with these entries that we can correct. In what follows we generate a function to get the first word of the street entry. We then count common entries by using the counter module.

In [9]:
def getting_norm_and_type_street_names(street_name):
    """Gets fisrt word of street - For checking type"""
    norm_street = None
    street_type = None
    #street_name = get_streets(element)
    #
    norm_street = normalizing_attributes(street_name)
    if street_name:
        street_type = norm_street.split()[0].strip()
        norm_street = norm_street
    return street_type, norm_street

In [10]:
type_street_counter = Counter(getting_norm_and_type_street_names(get_streets(element))[0] for element in get_element(SAMPLE_FILE, tags=('node', 'way', 'relation')))

In [11]:
type_street_counter

Counter({None: 6636,
         u'300': 1,
         u'7': 1,
         u'Alameda': 2,
         u'Auxiliar': 1,
         u'Av.': 1,
         u'Avenida': 146,
         u'Calle': 17,
         u'Carretera': 1,
         u'Colcabamba': 1,
         u'Coricancha': 1,
         u'Federico': 1,
         u'General': 1,
         u'Gozzoli': 1,
         u'Gran': 1,
         u'Guardia': 1,
         u'Inca': 1,
         u'Jesus': 1,
         u'Jiron': 29,
         u'Jorge': 1,
         u'Jose': 1,
         u'La': 1,
         u'Las': 4,
         u'Leoncio': 1,
         u'Los': 4,
         u'Manuel': 1,
         u'Maria': 1,
         u'Mercedes': 1,
         u'Monterrosa': 1,
         u'Pacasmayo': 1,
         u'Pasaje': 1,
         u'Paseo': 1,
         u'Paz': 1,
         u'Prolongacion': 4,
         u'Puente': 1,
         u'San': 2,
         u'Santa': 1,
         u'Satelite': 1,
         u'Tupac': 1,
         u'Victor': 1,
         u'maynas': 1})

From the list, we identify four problems:
1. Av., Avenida and Avienda: They all refer to the same stret type.
2. Defensores and Defendores: This might be a typing issue.
3. Jr., Jr, Jiron: They all refere to the same street type.
4. Calle and calle: difference because of capitalization.
We correct this discrepancies in the data below.

In [12]:
def corrections_street_type(street_type):
    """changing types according to corrections"""
    if street_type in ["Av", "Av.", "Avienda"]:
        correct_type = "Avenida"
    elif street_type in ["Defendores"]:
        correct_type = "Defensores"
    elif street_type in ["Jr", "Jr."]:
        correct_type = "Jiron"
    elif street_type in ["calle"]:
        correct_type = "Calle"
    else:
        correct_type = None
    return correct_type

In [13]:
def correcting_street_names(street_name):
    """"Corrects problematic entries according to specified rule in function
    corrections_street_type"""
    street_name, street_type = getting_norm_and_type_street_names(street_name)
    corrected_name = corrections_street_type(street_type)
    if corrected_name:
        street_name = street_name.replace(street_type, corrected_name, 1)
    return street_name

Below we print only the street types for our data set to see if changes were correctly applied. As it is possible to see, all changes specified by the function yield the expected result.

In [14]:
street_type_checker = set()
for element in get_element(SAMPLE_FILE, tags=('node', 'way', 'relation')):
    street_name = get_streets(element)
    if street_name:
        street_type_checker.add(correcting_street_names(street_name).split()[0])

In [15]:
street_type_checker

{u'300',
 u'7',
 u'Alameda',
 u'Auxiliar',
 u'Av.',
 u'Avenida',
 u'Calle',
 u'Carretera',
 u'Colcabamba',
 u'Coricancha',
 u'Federico',
 u'General',
 u'Gozzoli',
 u'Gran',
 u'Guardia',
 u'Inca',
 u'Jesus',
 u'Jiron',
 u'Jorge',
 u'Jose',
 u'La',
 u'Las',
 u'Leoncio',
 u'Los',
 u'Manuel',
 u'Maria',
 u'Mercedes',
 u'Monterrosa',
 u'Pacasmayo',
 u'Pasaje',
 u'Paseo',
 u'Paz',
 u'Prolongacion',
 u'Puente',
 u'San',
 u'Santa',
 u'Satelite',
 u'Tupac',
 u'Victor',
 u'maynas'}

# Checking amenities and tag contents for banks and fast food.

We wanted to make a more rigourous analysis of banks and fast foods in the data. We started checking what are the available amenity tags. From there we went to analyzing what type of data is associated with banks and fast food. We observed and try to correct problems realted with multiple names for a single entities and cases of bad tagging.

In [16]:
def get_amenity(element):
    amen_name = None
    type_amen = None
    for tag in element.findall('tag'):
        if tag.attrib['k'] == 'name':
                amen_name = tag.attrib['v']
        if tag.attrib['k'] =='amenity':
            type_amen = tag.attrib['v']
    return (normalizing_attributes(type_amen), normalizing_attributes(amen_name))
        
        

In [135]:
types_amen = set()
for element in get_element(OSM_FILE, tags=('node', 'way', 'relation')):
    types_amen.add(get_amenity(element)[0])

In [136]:
types_amen

{None,
 u'arts_centre',
 u'atm',
 u'bank',
 u'bar',
 u'bench',
 u'bicycle_parking',
 u'bicycle_rental',
 u'bicycle_repair_station',
 u'bureau_de_change',
 u'bus_station',
 u'cafe',
 u'car_rental',
 u'car_sharing',
 u'car_wash',
 u'casino',
 u'charging_station',
 u'childcare',
 u'cinema',
 u'clinic',
 u'clinic;doctors',
 u'clinic;parking',
 u'college',
 u'community_centre',
 u'concert_hall',
 u'courthouse',
 u'dancing_school',
 u'dentist',
 u'doctors',
 u'dojo',
 u'drinking_water',
 u'driving_school',
 u'embassy',
 u'events_venue',
 u'exhibition_centre',
 u'fast_food',
 u'ferry_terminal',
 u'fire_station',
 u'food_court',
 u'fountain',
 u'fuel',
 u'grave_yard',
 u'gym',
 u'hospital',
 u'ice_cream',
 u'internet_cafe',
 u'karaoke',
 u'kindergarten',
 u'language_school',
 u'library',
 u'love_hotel',
 u'marketplace',
 u'monastery',
 u'money_transfer',
 u'motorcycle_parking',
 u'music_school',
 u'nightclub',
 u'nursery',
 u'parking',
 u'parking_entrance',
 u'parking_space',
 u'payment_termin

In [137]:
bank_names = set()
for element in get_element(OSM_FILE, tags=('node', 'way', 'relation')):
    type_amen, amen_name = get_amenity(element)
    if type_amen == 'bank':
            bank_names.add(amen_name)

In [138]:
bank_names

{None,
 u'AFP Habitat',
 u'Acresip',
 u'Agente BCP',
 u'Agente Banco de la Nacion',
 u'Almacenes Del Banco De La Nacion',
 u'Arakaki',
 u'Azteca',
 u'BANBIF',
 u'BBVA',
 u'BBVA Banco Continental',
 u'BBVA Continental',
 u'BBVA Continetal',
 u'BCP',
 u'BCP Agencia Ebony',
 u'BCP Agente',
 u'BCP Contigo',
 u'BCP Interbank',
 u'BIF',
 u'Ban Bif',
 u'BanBIF',
 u'BanBif',
 u'Banbif',
 u'Banca Falabella',
 u'Banco Azteca',
 u'Banco Cencosud',
 u'Banco Central de Reserva del Peru',
 u'Banco Citibank',
 u'Banco Continental',
 u'Banco Edificar',
 u'Banco Edyficar',
 u'Banco Falabella',
 u'Banco Financiero',
 u'Banco GNB',
 u'Banco InterBank',
 u'Banco Interbank',
 u'Banco Internacional del Peru',
 u'Banco de Comercio',
 u'Banco de Credito',
 u'Banco de Credito BCP',
 u'Banco de Credito De Peta',
 u'Banco de Credito de Peru',
 u'Banco de Credito del Peru',
 u'Banco de Credito del Peru (BCP)',
 u'Banco de Credito del Peu',
 u'Banco de la Nacion',
 u'Banco del Comercio',
 u'Besco',
 u'Caja Arequip

In [21]:
def rechecker_names(dict_names):
    """creates a set of names from dict values"""
    checking_list = []
    for item in dict_names.values():
        checking_list.extend(item)
    return set(checking_list)

In [141]:
#Bank Clean    
bank_dict = {
        "BBVA": [],
        "BCP": [],
        "Banco de la Nacion": [],
        "Interbank": [],
        "GNB": [],
        "Falabella": [],
        "Azteca": [],
        "Mi Banco": [],
        "Financiero": [],
        "Scotiabank": [],
        "Edyficar": [],
        "Citibank": [],
        "BIF": [],
        "Compartamos": [],
        "Others": []
    }

for bank in bank_names:
    if bank:
        if "continental" in bank.lower() or "bbva" in bank.lower():
            bank_dict["BBVA"].append(bank)
        elif ("banco" in bank.lower() and "credito" in bank.lower()) or ("bcp" in bank.lower()):
            bank_dict["BCP"].append(bank)
        elif "mibanco" in bank.lower().replace(" ", ""):
            bank_dict["Mi Banco"].append(bank)
        elif ("interbank" in bank.lower() or "internacional" in bank.lower() 
              or "intel" in bank.lower()):
            bank_dict["Interbank"].append(bank)
        elif "nacion" in bank.lower() or "multired" in bank.lower():
            bank_dict["Banco de la Nacion"].append(bank)
        elif "gnb" in bank.lower():
            bank_dict["GNB"].append(bank)
        elif "financiero" in bank.lower():
            bank_dict["Financiero"].append(bank)
        elif "azteca" in bank.lower():
            bank_dict["Azteca"].append(bank)
        elif "falabella" in bank.lower():
            bank_dict["Falabella"].append(bank)
        elif "scotiabank" in bank.lower():
            bank_dict["Scotiabank"].append(bank)
        elif "edyficar" in bank.lower():
            bank_dict["Edyficar"].append(bank)
        elif "citi" in bank.lower():
            bank_dict["Citibank"].append(bank)
        elif "bif" in bank.lower():
            bank_dict["BIF"].append(bank)
        elif "compartamos" in bank.lower():
            bank_dict["Compartamos"].append(bank)
        else:
            bank_dict["Others"].append(bank)   

In [142]:
#Checking correctness of tag. Only choose those tags that we have already selected as important.
def dict_no_other(dictio_items): 
    selected_items = {item: value for item, value in dictio_items.items() if item != "Others"}
    return selected_items
selected_banks = dict_no_other(bank_dict)  


In [143]:
#Inverting bank dictionary so mapping is easier
def reverse_dict_items(dict_to_reverse):
    reverse_dict = {item: value for value, items in dict_to_reverse.items() for item in items}
    for value in set(reverse_dict.values()):
        if value not in reverse_dict.keys():
            reverse_dict[value] = value
    return reverse_dict

reverse_bank_dict = reverse_dict_items(selected_banks)
curated_bank_set = reverse_bank_dict.keys()
curated_bank_set

[u'Banco de Credito BCP',
 u'INTERBANK CENTRAL',
 u'Interbank',
 u'Banco de Credito',
 u'Banco Financiero',
 u'Banco de Credito del Peru',
 u'BANBIF',
 u'Cajero BCP Tottus',
 u'Compartamos Finaciera',
 u'banco BCP',
 u'Banco Continental',
 u'Compartamos Financeria',
 u'Cajero Multired Banco de la Nacion',
 u'Falabella',
 u'Banco Falabella',
 u'InterBank',
 u'BCP Contigo',
 u'Torre Banco de la Nacion',
 u'banco interbank',
 u'Banco de la Nacion',
 u'BIF',
 u'banco BBVA Continental',
 u'Banco Azteca',
 u'Mibanco',
 u'BBVA Banco Continental',
 u'Interbank - PlazaVea Corpac',
 u'Banco de Credito De Peta',
 u'Banco de Credito del Peru (BCP)',
 u'Banco de Credito de Peru',
 u'Banca Falabella',
 u'Agente BCP',
 u'MultiRed',
 u'Banco de Credito del Peu',
 'Financiero',
 u'Banco InterBank',
 u'BCP Agente',
 u'Azteca',
 u'Citibank',
 u'Financiera EDYFICAR',
 u'MiBanco',
 u'Banco Edyficar',
 u'BBVA',
 u'mibanco',
 u'Banco Interbank',
 u'BanBif',
 'GNB',
 u'Scotiabank - Centro de Credito Vehicular

In [144]:
# Normalizing bank names
def name_checker(element, reverse_dict, curated_list):
    type_amen, amen_name = get_amenity(element)
    if amen_name and amen_name in curated_list:
        amen_name = reverse_dict[amen_name]
    return amen_name

In [145]:

def bank_tag_checker(element):
    type_amen, amen_name = get_amenity(element)
    if amen_name and amen_name in curated_bank_set and not type_amen:
        type_amen = "bank"
    return type_amen

In [146]:
def bank_name_tag_checker(element):
    """Function for double checking if bank names and tags are Ok."""
    type_amen = bank_tag_checker(element)
    amen_name = name_checker(element, reverse_bank_dict, curated_bank_set)
    return type_amen, amen_name

Here we check that our function is running as expected. As it is possible to see in the output below, the code is working as expected. For the banks that we have, it corrects the tag and labels it as a bank.

In [147]:
for element in get_element(SAMPLE_FILE, tags=('node', 'way', 'relation')):
    tag, name = bank_name_tag_checker(element)
    if tag == "bank" and name:
        print tag, name
        

bank Interbank
bank BBVA
bank Interbank
bank Scotiabank
bank BIF
bank Scotiabank
bank Mi Banco


## Checking fast food names

Here we check the main differences with fast food names. First we get the list of fast foods and check for different instances that refer to the same restaurant. We correct the list to modify these inconsistencies. We compare this list to the list of restaurants to check if there are restaurants that are double listed as restaurants and fast food. If this is the case, we include them in the fast food and take it off from the restaurant list.

In [148]:
#Checking fast food restaurants
fast_food_set = set()
for element in get_element(OSM_FILE, tags=('node', 'way', 'relation')):
    type_amen, amen_name = get_amenity(element)
    if type_amen=="fast_food" or type_amen == "restaurant":
            fast_food_set.add(amen_name) 

In [149]:
fast_food_set

{u'Chifa Cam-Lu',
 u'Chobis',
 u'Asoc.de Viv.Chillon Puente Piedra/Sonidos Patagonia',
 u'chino ELECTRI',
 u'Chifa Fu In',
 u'Alexandra',
 u'Polleria El Cavilan',
 u"Buffet's Castaneda",
 u'Chifa Tong',
 u'restaurant Lawrence',
 u"Bembo's",
 u'Chicharrones Tobi II',
 u'Norkys',
 u'Milagritos',
 u'El Rocoto,,Miraflores',
 u'La Gran Estacion',
 u'Polleria',
 u'Salon Juanita',
 u'Il Pastifico',
 u'Chifa Las Flores',
 u'Restaurante',
 u'restaurante Malicha',
 u'Puerto Azli',
 u'Chifa Fet Yan',
 u'Chifa Bejing',
 u'Capirena',
 u'Madan Tusam',
 u'Don Arturo',
 u'Chifa Bet Jing',
 u'Punta Sal',
 u"D'Lalo Restaurant",
 u'El Sonido del Silencio',
 u'El Plombino',
 u'Aroma & Gusto',
 u"Papacho's",
 u'Garay',
 u'Merlin',
 u'Las Nortenitas',
 u"Cevicheria Johmy's",
 u"D' Luckys",
 u'El Gustazo',
 u'Sabor a Mar',
 u"Mc Donald's",
 u'Chifa Men Wha',
 u'Chifa Chung Wha',
 u'Tragamonedas',
 u'Pihuicho',
 u'Cabo Azul',
 u'Piura Amazonas',
 u'Chifa Shun De',
 u"Topi's",
 u'Central de Restaurantes',
 u'R

In [150]:
#Bank Clean    
fast_food_dict = {
        "KFC": [],
        "Bembos": [],
        "Burger King": [],
        "Pizza Hut": [],
        "Papa John's": [],
        "Popeye's": [],
        "Norky's": [],
        "Rocky's": [],
        "Subway": [],
        "McDonald's": [],
        "China Wok": [],
        "D'nnos Pizza": [],
        "Telepizza": [],
        "Tip Top": [],
        "Dunkin Donuts": [],
        "Domino's Pizza": [],
        "Others": []
    }

def renaming_fast_food(fast_food_list): 
    for fast_food in fast_food_list:
        if fast_food:
            if "bembo" in fast_food.lower():
                fast_food_dict["Bembos"].append(fast_food)
            if "kfc" in fast_food.lower() or "kentucky" in fast_food.lower():
                fast_food_dict["KFC"].append(fast_food)
            if "burger king" in fast_food.lower():
                fast_food_dict["Burger King"].append(fast_food)
            if "pizza hut" in fast_food.lower():
                fast_food_dict["Pizza Hut"].append(fast_food)
            if "papa john" in fast_food.lower():
                fast_food_dict["Papa John's"].append(fast_food)
            if "popeye" in fast_food.lower():
                fast_food_dict["Popeye's"].append(fast_food)
            if "norky" in fast_food.lower():
                fast_food_dict["Norky's"].append(fast_food)
            if "rocky" in fast_food.lower() or "roky" in fast_food.lower() :
                fast_food_dict["Rocky's"].append(fast_food)
            if "subway" in fast_food.lower():
                fast_food_dict["Subway"].append(fast_food)
            if "mcdonald" in fast_food.lower() or "mc don" in fast_food.lower():
                fast_food_dict["McDonald's"].append(fast_food)
            if "china wok" in fast_food.lower() or "chinawok" in fast_food.lower():
                fast_food_dict["China Wok"].append(fast_food)
            if "d'nnos " in fast_food.lower() or "dinnos" in fast_food.lower():
                fast_food_dict["D'nnos Pizza"].append(fast_food)
            if "telepizz" in fast_food.lower():
                fast_food_dict["Telepizza"].append(fast_food)
            if "tip top" in fast_food.lower():
                fast_food_dict["Tip Top"].append(fast_food)
            if "dunki" in fast_food.lower():
                fast_food_dict["Dunkin Donuts"].append(fast_food)
            if "domino" in fast_food.lower():
                fast_food_dict["Domino's Pizza"].append(fast_food)
            else:
                fast_food_dict["Others"].append(fast_food)
    return fast_food_dict

fast_food_dict = renaming_fast_food(fast_food_set)
fast_food_dict["KFC"].append("KfC")
fast_food_dict

{'Bembos': [u"Bembo's",
  u'Bembos Plaza Vea Callao',
  u'Bembos Bellavista',
  u'Bembos Minka',
  u'Bembos'],
 'Burger King': [u'Burger King'],
 'China Wok': [u'Chinawok', u'China Wok'],
 "D'nnos Pizza": [u"D'nnos Pizza"],
 "Domino's Pizza": [u"Domino's Pizza Minka",
  u"Domino's Pizza",
  u"Domino's Pizza La Punta"],
 'Dunkin Donuts': [u'Dunki Donuts'],
 'KFC': [u'KFC/Pizza Hut', u'KFC / Pizza Hut', u'KFC', 'KfC'],
 "McDonald's": [u"Mc Donald's", u"McDonald's", u'McDonalds'],
 "Norky's": [u'Norkys',
  u'Polleria Norkys',
  u"Polleria Norky's",
  u"Norky's",
  u'Polleria Norky\xb4s',
  u"Norky's Criollo"],
 'Others': [u'Chifa Cam-Lu',
  u'Chobis',
  u'Asoc.de Viv.Chillon Puente Piedra/Sonidos Patagonia',
  u'chino ELECTRI',
  u'Chifa Fu In',
  u'Alexandra',
  u'Polleria El Cavilan',
  u"Buffet's Castaneda",
  u'Chifa Tong',
  u'restaurant Lawrence',
  u"Bembo's",
  u'Chicharrones Tobi II',
  u'Norkys',
  u'Milagritos',
  u'El Rocoto,,Miraflores',
  u'La Gran Estacion',
  u'Polleria',


In [151]:
#Reversing restaurant names
selected_rest = dict_no_other(fast_food_dict) 
reverse_rest_dict = reverse_dict_items(selected_rest)
curated_rest = reverse_rest_dict.keys()

In [152]:
#Changing Nonen and restaurant tag to fast food tag
def fast_food_tag_checker(element):
    type_amen, amen_name = get_amenity(element)
    if amen_name and amen_name in curated_rest and type_amen=='restaurant':
        type_amen = "fast_food"
    elif amen_name and amen_name in curated_rest and not type_amen:
        type_amen = "fast_food"
    return type_amen

In [153]:
def rest_name_tag_checker(element):
    """Function for checking if tag and names match specifications"""
    type_amen = fast_food_tag_checker(element)
    amen_name = name_checker(element, reverse_rest_dict, curated_rest)
    return type_amen, amen_name

In [154]:
for element in get_element(SAMPLE_FILE, tags=('node', 'way', 'relation')):
    tag, amen_name = get_amenity(element)
    if tag == "fast_food" or tag == "restaurant":
        print rest_name_tag_checker(element)

(u'restaurant', u'Polleria Hikari')
('fast_food', "Rocky's")
(u'fast_food', u'Chifa Zhen')
(u'restaurant', u'Pizzeria El Italiano')
(u'restaurant', u'Perroquet')
(u'restaurant', u'EDO Sushi Bar')
(u'restaurant', u'Combo Marino')
(u'restaurant', u'Villa Chicken')
(u'fast_food', 'KFC')
('fast_food', "Norky's")
(u'restaurant', u'Palco Restobar')
(u'restaurant', u'Los Incas')
(u'restaurant', u'Mr. Sushi')
(u'restaurant', u'OiSHii')
(u'restaurant', None)
(u'restaurant', u'Restaurant Vita')
(u'restaurant', u'La Diabla')
(u'restaurant', u'Osharz')
(u'restaurant', u'Chifa Hong Yi')
(u'restaurant', u'Camila')
(u'restaurant', u'Chifa Sabor Cantones')
(u'restaurant', u'El Mistiano')
(u'restaurant', u'Restaurant Buen Sabor')
(u'restaurant', u'Marcelino Pizza & Vino')
(u'restaurant', u"Mr. Noky's")
(u'restaurant', u'Las Canastas')
(u'restaurant', u'Caleo')
(u'restaurant', u'Sansushito')
(u'restaurant', u'Chifa Fu Hua')
(u'fast_food', 'KFC')
(u'restaurant', u'Pollos Chan')
(u'restaurant', u'La Panka

# Passing data to CSV

Having written our functions that will help us clean the data, we pass the data to the appropriate CSV files, checking consistency.

In order to pass our data to csv files, we use this code. We update it appropriately so the changes introduced by our defined functions are used. 

In [155]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus

import schema



OSM_PATH = "lima_peru.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



def tag_info_getter(tag, element):
    if PROBLEMCHARS.search(tag.attrib['k']):
        return None
    else:
        dictio_tag = {}
        dictio_tag['id'] = element.attrib['id']
        k_value_split = tag.attrib['k'].split(":")
        if len(k_value_split) > 1:
            key_dict = ":".join(k_value_split[:-1])
            type_dict = k_value_split[-1]
            dictio_tag['key'] = key_dict
            dictio_tag['type'] = type_dict
        else:
            key_dict = tag.attrib['k']
            type_dict = 'regular'
            dictio_tag['key'] = tag.attrib['k']
            dictio_tag['type'] = type_dict
        if key_dict.startswith("addr") or type_dict == "street":
            dictio_tag['value'] = correcting_street_names(tag.attrib['v'])
        norm_values = normalizing_attributes(tag.attrib['v'])
        dictio_tag['value'] = norm_values
        if tag.attrib['k'] == "amenity":
            dictio_tag['value'] = bank_tag_checker(element)
            if dictio_tag['value'] == tag.attrib['v']:
                dictio_tag['value'] = fast_food_tag_checker(element)
        if tag.attrib['k'] == "name":
            dictio_tag['value'] = name_checker(element, reverse_bank_dict, curated_bank_set)
            if dictio_tag['value'] == tag.attrib['v']:
                dictio_tag['value'] = name_checker(element, reverse_rest_dict, curated_rest)
        return dictio_tag

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}

    # YOUR CODE HERE
    if element.tag == 'node':
        node_attribs['id'] = element.attrib['id']
        node_attribs['uid'] = element.attrib['uid']
        node_attribs['user'] = element.attrib['user']
        node_attribs['version'] = element.attrib['version']
        node_attribs['lat'] = element.attrib['lat']
        node_attribs['lon'] = element.attrib['lon']
        node_attribs['timestamp'] = element.attrib['timestamp']
        node_attribs['changeset'] = element.attrib['changeset']
        tags = [] 
        for tag in element.findall('tag'):
            dictio_tag = tag_info_getter(tag, element)
            tags.append(dictio_tag)
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        way_attribs['id'] = element.attrib['id']
        way_attribs['user'] = element.attrib['user']
        way_attribs['version'] = element.attrib['version']
        way_attribs['uid'] = element.attrib['uid']
        way_attribs['timestamp'] = element.attrib['timestamp']
        way_attribs['changeset'] = element.attrib['changeset']
        tags = []
        way_nodes = []
        counter = 0
        for node in element.findall('nd'):
            dictio_nodes = {}
            dictio_nodes['id'] = element.attrib['id']
            dictio_nodes['node_id'] = node.attrib['ref']
            dictio_nodes['position'] = counter
            counter += 1
            way_nodes.append(dictio_nodes)
        for tag in element.findall('tag'):
            dictio_tag = tag_info_getter(tag, element)
            tags.append(dictio_tag)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                try:
                    if validate is True:
                        if "node_tags" in el.keys():
                            if None in el["node_tags"]:
                                el["node_tags"].remove(None)
                        elif "way_tags" in el.keys():
                            if None in el["way_tags"]:
                                el["way_tags"].remove(None)                        
                        validate_element(el, validator)
                except AttributeError:
                    print el
                    break

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


#if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
process_map(OSM_FILE, validate=True)